# Epidemiological Surveillance and Outbreak Detection

**Tier 2: AWS Starter Project**

This notebook demonstrates:
1. Generating synthetic disease surveillance data
2. Uploading to AWS S3
3. Triggering Lambda analysis
4. Querying results from DynamoDB
5. Visualizing epidemic curves and outbreak patterns
6. SIR model simulation
7. R0 estimation and analysis

**Author:** Research Jumpstart  
**Last Updated:** 2025-11-14

## Setup and Imports

In [ ]:
# Standard library
import json
import time
from datetime import datetime, timedelta

# AWS SDK
import boto3

# Visualization
import matplotlib.pyplot as plt

# Data manipulation
import numpy as np
import pandas as pd
import seaborn as sns
from botocore.exceptions import ClientError

# Scientific computing
from scipy import stats
from scipy.integrate import odeint

# Configuration
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)

print("✓ Imports complete")

## AWS Configuration

Set your AWS resource names here:

In [ ]:
# AWS Configuration - UPDATE THESE VALUES
S3_BUCKET = "epidemiology-data-XXXXXX"  # Replace with your bucket name
DYNAMODB_TABLE = "DiseaseReports"
LAMBDA_FUNCTION = "analyze-disease-surveillance"

# AWS clients
s3_client = boto3.client("s3")
dynamodb = boto3.resource("dynamodb")
lambda_client = boto3.client("lambda")

# Verify bucket access
try:
    s3_client.head_bucket(Bucket=S3_BUCKET)
    print(f"✓ Connected to S3 bucket: {S3_BUCKET}")
except ClientError as e:
    print(f"❌ Error accessing bucket: {e}")
    print("Please update S3_BUCKET variable with your bucket name")

## 1. Generate Synthetic Disease Surveillance Data

Create realistic case report data with an outbreak signal

In [ ]:
def generate_case_data(n_cases=500, outbreak_start_day=30, outbreak_magnitude=3.0):
    """
    Generate synthetic disease case data.

    Args:
        n_cases: Total number of cases
        outbreak_start_day: Day when outbreak begins
        outbreak_magnitude: Multiplier for outbreak intensity

    Returns:
        DataFrame with case data
    """
    np.random.seed(42)

    # Generate dates over 60 days
    start_date = datetime(2024, 1, 1)
    days = 60

    # Baseline case probability (Poisson-like)
    baseline_rate = 5  # cases per day

    # Generate daily case counts with outbreak signal
    daily_cases = []
    for day in range(days):
        if day < outbreak_start_day:
            # Baseline period
            rate = baseline_rate
        else:
            # Outbreak period with exponential growth then decay
            days_since_outbreak = day - outbreak_start_day
            if days_since_outbreak < 15:
                # Growth phase
                rate = baseline_rate * (1 + outbreak_magnitude * (days_since_outbreak / 15))
            else:
                # Decay phase
                decay_days = days_since_outbreak - 15
                rate = baseline_rate * outbreak_magnitude * np.exp(-0.1 * decay_days)

        cases = max(0, int(np.random.poisson(rate)))
        daily_cases.append(cases)

    # Generate individual case records
    cases = []
    case_counter = 1

    regions = ["northeast", "southeast", "midwest", "southwest", "west"]
    age_groups = ["0-4", "5-14", "15-24", "25-34", "35-44", "45-54", "55-64", "65-74", "75+"]
    sexes = ["M", "F"]
    outcomes = ["recovered", "hospitalized", "icu", "fatal", "active"]
    diseases = ["influenza", "covid-19", "measles", "pertussis"]

    # Weights for outcomes (most recover)
    outcome_weights = [0.85, 0.10, 0.03, 0.01, 0.01]

    for day_idx, day_cases in enumerate(daily_cases):
        report_date = start_date + timedelta(days=day_idx)

        for _ in range(day_cases):
            # Hotspot region during outbreak
            if day_idx >= outbreak_start_day:
                region = np.random.choice(regions, p=[0.5, 0.2, 0.15, 0.10, 0.05])
            else:
                region = np.random.choice(regions)

            case = {
                "case_id": f"case_{case_counter:06d}",
                "disease": np.random.choice(diseases, p=[0.7, 0.2, 0.05, 0.05]),
                "report_date": report_date.strftime("%Y-%m-%d"),
                "symptom_onset_date": (
                    report_date - timedelta(days=np.random.randint(0, 7))
                ).strftime("%Y-%m-%d"),
                "region": region,
                "zip_code": f"{np.random.randint(10000, 99999)}",
                "age_group": np.random.choice(age_groups),
                "sex": np.random.choice(sexes),
                "outcome": np.random.choice(outcomes, p=outcome_weights),
            }

            cases.append(case)
            case_counter += 1

    df = pd.DataFrame(cases)
    return df


# Generate data
case_data = generate_case_data(n_cases=500, outbreak_start_day=30, outbreak_magnitude=3.0)

print(f"Generated {len(case_data)} case records")
print(f"Date range: {case_data['report_date'].min()} to {case_data['report_date'].max()}")
print("\nFirst 5 records:")
case_data.head()

### Visualize Generated Data

In [ ]:
# Daily case counts
daily_counts = case_data.groupby("report_date").size()

plt.figure(figsize=(14, 6))
plt.plot(pd.to_datetime(daily_counts.index), daily_counts.values, marker="o", linewidth=2)
plt.axvline(x=pd.to_datetime("2024-01-31"), color="red", linestyle="--", label="Outbreak Start")
plt.xlabel("Date", fontsize=12)
plt.ylabel("Daily Cases", fontsize=12)
plt.title("Synthetic Epidemic Curve - Daily Case Counts", fontsize=14, fontweight="bold")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Baseline period (days 1-30): mean = {daily_counts.iloc[:30].mean():.1f} cases/day")
print(f"Outbreak period (days 31-60): mean = {daily_counts.iloc[30:].mean():.1f} cases/day")

## 2. Upload Data to S3

Upload case data to trigger Lambda processing

In [ ]:
# Save to CSV
csv_filename = f"case_reports_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
case_data.to_csv(csv_filename, index=False)

print(f"Saved case data to: {csv_filename}")

# Upload to S3
s3_key = f"case-data/{csv_filename}"

try:
    s3_client.upload_file(csv_filename, S3_BUCKET, s3_key)
    print(f"✓ Uploaded to s3://{S3_BUCKET}/{s3_key}")
    print("\nLambda function will automatically process this file.")
    print("Processing typically takes 10-30 seconds...")
except ClientError as e:
    print(f"❌ Upload error: {e}")

## 3. Wait for Lambda Processing

Monitor Lambda execution

In [ ]:
# Wait for Lambda to process
print("Waiting for Lambda processing...")
time.sleep(15)

# Check CloudWatch logs (recent)
logs_client = boto3.client("logs")
log_group = f"/aws/lambda/{LAMBDA_FUNCTION}"

try:
    # Get recent log streams
    response = logs_client.describe_log_streams(
        logGroupName=log_group, orderBy="LastEventTime", descending=True, limit=1
    )

    if response["logStreams"]:
        stream_name = response["logStreams"][0]["logStreamName"]

        # Get recent log events
        log_events = logs_client.get_log_events(
            logGroupName=log_group, logStreamName=stream_name, limit=20
        )

        print("\nRecent Lambda logs:")
        print("=" * 60)
        for event in log_events["events"][-10:]:
            print(event["message"].strip())
        print("=" * 60)

except Exception as e:
    print(f"Note: Could not retrieve logs: {e}")
    print("You can check logs manually in AWS CloudWatch console")

print("\n✓ Processing should be complete")

## 4. Query Results from DynamoDB

Retrieve analysis results

In [ ]:
from boto3.dynamodb.conditions import Attr

# Connect to table
table = dynamodb.Table(DYNAMODB_TABLE)

# Query summary records
response = table.scan(FilterExpression=Attr("record_type").eq("summary"), Limit=10)

summaries = response.get("Items", [])

print(f"Found {len(summaries)} summary records\n")

if summaries:
    # Get most recent summary
    latest_summary = max(summaries, key=lambda x: x["report_date"])

    print("=" * 60)
    print("LATEST ANALYSIS SUMMARY")
    print("=" * 60)
    print(f"Report Date: {latest_summary['report_date']}")
    print(f"Total Cases: {latest_summary['total_cases']}")
    print(f"Incidence Rate: {latest_summary['incidence_rate']:.2f} per 100,000")
    print(f"Outbreak Detected: {latest_summary['outbreak_detected']}")
    print(f"Outbreak Confidence: {latest_summary['outbreak_confidence']}")

    if "r0_estimate" in latest_summary:
        print(f"R0 Estimate: {latest_summary['r0_estimate']:.2f}")

    print("\n" + "=" * 60)

    # Parse detailed metrics
    if "metrics" in latest_summary:
        metrics = json.loads(latest_summary["metrics"])
        print("\nDETAILED METRICS:")
        print(f"  Case Fatality Rate: {metrics.get('case_fatality_rate', 0):.2f}%")
        print(f"  Hospitalization Rate: {metrics.get('hospitalization_rate', 0):.2f}%")
        print(f"  Average Daily Cases: {metrics.get('avg_daily_cases', 0):.1f}")
        print(f"  Max Daily Cases: {metrics.get('max_daily_cases', 0)}")

    # Parse outbreak signals
    if "outbreak_signals" in latest_summary:
        signals = json.loads(latest_summary["outbreak_signals"])
        print("\nOUTBREAK SIGNALS:")
        for reason in signals.get("reasons", []):
            print(f"  • {reason}")

        if "z_score" in signals:
            print(f"\n  Z-score: {signals['z_score']:.2f}")

        if "hotspot_region" in signals:
            print(f"  Hotspot Region: {signals['hotspot_region']}")
else:
    print("No summary records found yet.")
    print("Lambda may still be processing or there was an error.")
    print("Check CloudWatch logs for details.")

## 5. Visualize Epidemic Curve from DynamoDB

In [ ]:
if summaries and "epi_curve" in latest_summary:
    # Parse epidemic curve data
    epi_curve = json.loads(latest_summary["epi_curve"])

    # Convert to DataFrame
    epi_df = pd.DataFrame(epi_curve)
    epi_df["date"] = pd.to_datetime(epi_df["date"])

    # Plot
    plt.figure(figsize=(14, 6))
    plt.bar(epi_df["date"], epi_df["cases"], width=0.8, alpha=0.7, color="steelblue")
    plt.xlabel("Date", fontsize=12)
    plt.ylabel("Cases", fontsize=12)
    plt.title("Epidemic Curve from Lambda Analysis", fontsize=14, fontweight="bold")
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3, axis="y")
    plt.tight_layout()
    plt.show()

    print(f"Total cases in curve: {epi_df['cases'].sum()}")
    print(f"Peak date: {epi_df.loc[epi_df['cases'].idxmax(), 'date'].strftime('%Y-%m-%d')}")
    print(f"Peak cases: {epi_df['cases'].max()}")
else:
    print("No epidemic curve data available")

## 6. Geographic Analysis

Analyze disease distribution by region

In [ ]:
# Query case records by region
response = table.scan(FilterExpression=Attr("record_type").eq("case"), Limit=1000)

cases = response.get("Items", [])

if cases:
    cases_df = pd.DataFrame(cases)

    # Cases by region
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))

    # Region distribution
    region_counts = cases_df["region"].value_counts()
    axes[0].bar(region_counts.index, region_counts.values, color="coral", alpha=0.7)
    axes[0].set_xlabel("Region", fontsize=12)
    axes[0].set_ylabel("Case Count", fontsize=12)
    axes[0].set_title("Cases by Region", fontsize=13, fontweight="bold")
    axes[0].tick_params(axis="x", rotation=45)

    # Age group distribution
    age_counts = cases_df["age_group"].value_counts()
    axes[1].barh(age_counts.index, age_counts.values, color="teal", alpha=0.7)
    axes[1].set_xlabel("Case Count", fontsize=12)
    axes[1].set_ylabel("Age Group", fontsize=12)
    axes[1].set_title("Cases by Age Group", fontsize=13, fontweight="bold")

    plt.tight_layout()
    plt.show()

    print(f"\nAnalyzed {len(cases_df)} case records")
    print("\nRegional distribution:")
    print(region_counts.to_string())
else:
    print("Using original case data for geographic analysis...")

    # Use original data
    region_counts = case_data["region"].value_counts()

    plt.figure(figsize=(10, 6))
    plt.bar(region_counts.index, region_counts.values, color="coral", alpha=0.7)
    plt.xlabel("Region", fontsize=12)
    plt.ylabel("Case Count", fontsize=12)
    plt.title("Cases by Region", fontsize=14, fontweight="bold")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## 7. SIR Model Simulation

Simulate disease dynamics using Susceptible-Infected-Recovered model

In [ ]:
def sir_model(y, t, beta, gamma):
    """
    SIR model differential equations.

    Args:
        y: [S, infected_pop, R] current state
        t: time
        beta: transmission rate
        gamma: recovery rate

    Returns:
        [dS/dt, dI/dt, dR/dt]
    """
    S, infected_pop, R = y
    N = S + infected_pop + R

    dS = -beta * S * infected_pop / N
    dI = beta * S * infected_pop / N - gamma * infected_pop
    dR = gamma * infected_pop

    return [dS, dI, dR]


# Parameters
N = 100000  # Population
I0 = 10  # Initial infected
R0_value = 2.5  # Basic reproductive number
infectious_period = 5  # days

gamma = 1 / infectious_period
beta = R0_value * gamma

# Initial conditions
S0 = N - I0
R0_init = 0
y0 = [S0, I0, R0_init]

# Time vector (days)
t = np.linspace(0, 200, 200)

# Solve ODE
solution = odeint(sir_model, y0, t, args=(beta, gamma))
S, infected_pop, R = solution.T

# Plot
plt.figure(figsize=(14, 6))
plt.plot(t, S / N, label="Susceptible", linewidth=2, color="blue")
plt.plot(t, infected_pop / N, label="Infected", linewidth=2, color="red")
plt.plot(t, R / N, label="Recovered", linewidth=2, color="green")
plt.xlabel("Time (days)", fontsize=12)
plt.ylabel("Proportion of Population", fontsize=12)
plt.title(f"SIR Model Simulation (R0 = {R0_value})", fontsize=14, fontweight="bold")
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Peak infection
peak_idx = np.argmax(infected_pop)
print(f"\nSIR Model Results (R0 = {R0_value}):")
print(f"Peak infections: {int(infected_pop[peak_idx])} ({infected_pop[peak_idx] / N * 100:.1f}% of population)")
print(f"Peak day: {int(t[peak_idx])}")
print(f"Final attack rate: {R[-1] / N * 100:.1f}% of population")
print(f"Total affected: {int(R[-1])} people")


## 8. R0 Estimation from Data

Estimate reproductive number from case data

In [ ]:
def estimate_r0_from_growth(case_data, serial_interval=3.0):
    """
    Estimate R0 using exponential growth method.

    Args:
        case_data: DataFrame with cases
        serial_interval: Mean time between infections (days)

    Returns:
        R0 estimate and confidence interval
    """
    # Daily cases
    case_data["report_date"] = pd.to_datetime(case_data["report_date"])
    daily_cases = case_data.groupby("report_date").size()

    # Early growth phase (first 20 days of outbreak)
    outbreak_start = pd.to_datetime("2024-01-31")
    growth_end = outbreak_start + timedelta(days=20)

    growth_phase = daily_cases[
        (daily_cases.index >= outbreak_start) & (daily_cases.index <= growth_end)
    ]

    if len(growth_phase) < 5:
        return None

    # Log-linear regression
    growth_phase_nonzero = growth_phase[growth_phase > 0]
    x = np.arange(len(growth_phase_nonzero))
    y = np.log(growth_phase_nonzero.values)

    slope, _intercept, r_value, _p_value, std_err = stats.linregress(x, y)

    # R0 calculation
    growth_rate = slope
    r0 = 1 + (growth_rate * serial_interval)

    # Confidence interval
    r0_lower = 1 + ((growth_rate - 1.96 * std_err) * serial_interval)
    r0_upper = 1 + ((growth_rate + 1.96 * std_err) * serial_interval)

    return {
        "r0": max(0, r0),
        "r0_lower": max(0, r0_lower),
        "r0_upper": max(0, r0_upper),
        "growth_rate": growth_rate,
        "r_squared": r_value**2,
    }


# Estimate R0
r0_result = estimate_r0_from_growth(case_data)

if r0_result:
    print("=" * 60)
    print("R0 ESTIMATION FROM CASE DATA")
    print("=" * 60)
    print(f"R0 estimate: {r0_result['r0']:.2f}")
    print(f"95% CI: [{r0_result['r0_lower']:.2f}, {r0_result['r0_upper']:.2f}]")
    print(f"Growth rate: {r0_result['growth_rate']:.3f} per day")
    print(f"R-squared: {r0_result['r_squared']:.3f}")
    print("\nInterpretation:")
    if r0_result["r0"] > 1:
        print(f"  R0 > 1: Outbreak will grow (each case infects ~{r0_result['r0']:.1f} others)")
    elif r0_result["r0"] == 1:
        print("  R0 = 1: Endemic equilibrium")
    else:
        print("  R0 < 1: Outbreak will decline")
    print("=" * 60)
else:
    print("Insufficient data for R0 estimation")

## 9. Compare Multiple R0 Scenarios

Visualize how different R0 values affect outbreak dynamics

In [ ]:
# Compare different R0 values
r0_values = [1.5, 2.0, 2.5, 3.0]
colors = ["blue", "green", "orange", "red"]

plt.figure(figsize=(14, 6))

for r0_val, color in zip(r0_values, colors):
    beta = r0_val * gamma
    solution = odeint(sir_model, y0, t, args=(beta, gamma))
    S, infected_pop, R = solution.T

    plt.plot(t, infected_pop, label=f"R0 = {r0_val}", linewidth=2, color=color)

plt.xlabel("Time (days)", fontsize=12)
plt.ylabel("Number of Infected", fontsize=12)
plt.title("Effect of R0 on Outbreak Trajectory", fontsize=14, fontweight="bold")
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nR0 values for reference:")
print("  R0 = 1.5: Moderate transmission (e.g., influenza)")
print("  R0 = 2.0-2.5: COVID-19 (original strain)")
print("  R0 = 3.0+: High transmission (e.g., measles)")


## 10. Cost Analysis

Estimate AWS costs for this analysis

In [ ]:
# Cost estimation
costs = {
    "S3 Storage (5 MB, 1 week)": 0.00,
    "S3 PUT requests (10)": 0.00,
    "Lambda invocations (5 @ 15s)": 0.05,
    "DynamoDB writes (500)": 0.01,
    "DynamoDB reads (100)": 0.00,
    "SNS notifications (1)": 0.00,
    "Data transfer (10 MB)": 0.00,
}

total_cost = sum(costs.values())

print("=" * 60)
print("AWS COST ESTIMATE FOR THIS SESSION")
print("=" * 60)
for service, cost in costs.items():
    print(f"{service:.<50} ${cost:.2f}")
print("-" * 60)
print(f"{'TOTAL ESTIMATED COST':.<50} ${total_cost:.2f}")
print("=" * 60)
print("\nNote: Most costs covered by AWS Free Tier for first 12 months.")
print("Typical project cost (including multiple runs): $6-10")

## Summary

### What We Accomplished

1. ✅ Generated synthetic disease surveillance data with outbreak signal
2. ✅ Uploaded data to S3, triggering Lambda analysis
3. ✅ Lambda calculated epidemiological metrics and detected outbreak
4. ✅ Results stored in DynamoDB for querying
5. ✅ Visualized epidemic curves and geographic patterns
6. ✅ Simulated SIR model dynamics
7. ✅ Estimated R0 from case data
8. ✅ Compared outbreak scenarios

### Key Epidemiological Insights

- **Outbreak Detection**: Statistical methods identified outbreak signal
- **R0 Estimation**: Quantified transmissibility
- **Geographic Clustering**: Identified hotspot regions
- **SIR Model**: Predicted outbreak trajectory

### Next Steps

1. **Extend Analysis**: Add more sophisticated outbreak detection algorithms
2. **Real Data**: Integrate with public health datasets (CDC, WHO)
3. **Advanced Models**: SEIR, SEIRS, age-structured models
4. **Forecasting**: Prophet, ARIMA for case prediction
5. **Move to Tier 3**: Production CloudFormation deployment

### Cleanup

Remember to clean up AWS resources to avoid charges:
```bash
# See cleanup_guide.md for detailed instructions
```